# Защита модели от Adversarial примеров

# ДАННАЯ РАБОТА РАССМАТРИВАЕТ НЕУДАЧНЫЙ ЭКСПЕРИМЕНТ

Данная работа является продолжением предыдущей, в которой рассматривалось создание Adversarial примеров в задаче классификации изображений.

Атака оказалась крайне удачной, и в ходе данной работы будут рассмотрены способы это исправить

## Основные способы защиты, рассмотренные в работе:
- [Ensemble adversarial training](https://arxiv.org/pdf/1705.07204) - модификация adversarial training, в которой adv. examples генерируются несколькими другими моделями
- нормализация входных данных
- использование небольшого dropout

Для начала зададим все необходимые параметры

In [64]:
import numpy as np
import random
import torch
from pathlib import Path

device = "cuda" if torch.cuda.is_available() else "cpu"
root   = Path("datasets/svhn_cls")
BATCH  = 128
EPOCHS = 100
BASE_LR = 3e-4
EPS    = 1/255
ADV_FRACTION = 0.25
MODELS_PATH = Path("models")
MODELS_PATH.mkdir(parents=True, exist_ok=True)
SEED = 17
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
device

'cuda'

Для нормализации нам нужно посчитать среднее и дисперсию нашего датасета. Заметим, что они считаются только по обучающей выборке, чтобы тестовая выборка никак не влияла на процесс обучения

Загрузим датасет, создадим трансформеры для него (с целью нормализации)

In [40]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

root = Path("datasets/svhn_cls")
mean = (0.4377, 0.4438, 0.4728)
std  = (0.1201, 0.1231, 0.1052)

tf_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
tf_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

ds_train = datasets.ImageFolder(root / "train", transform=tf_train)
ds_test  = datasets.ImageFolder(root / "test",  transform=tf_test)
dl_train = DataLoader(ds_train, batch_size=128, shuffle=True,  num_workers=4, pin_memory=True)
dl_test  = DataLoader(ds_test,  batch_size=256, shuffle=False, num_workers=4, pin_memory=True)

Создадим наш ансамбль моделей, которые будут генерировать adv. examples

Для этого возьмём готовый resnet и за finetune-им его на нашем датасете (после этого представлен кусок кода для использования готовых дообученных моделей из [диска](https://disk.360.yandex.ru/d/PZcDdBJIB4_P8A)

In [ ]:
import copy, random, numpy as np, torch
import torch.nn.functional as F
from torchvision import models

# ─── helpers ────────────────────────────────────────────────────────────────────
def eval_accuracy(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x).argmax(1)
            correct += (pred == y).sum().item()
            total   += y.size(0)
    return correct / total

# ─── main training routine ──────────────────────────────────────────────────────
def train_resnet(seed: int,
                 epochs: int = 4,
                 lr: float   = 3e-4,
                 wd: float   = 5e-4):
    torch.manual_seed(seed); np.random.seed(seed); random.seed(seed)

    net = models.resnet18(weights="IMAGENET1K_V1")
    net.fc = nn.Linear(net.fc.in_features, 10)
    net.to(device)

    opt   = torch.optim.AdamW(net.parameters(), lr=lr, weight_decay=wd)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(
                opt, T_max=epochs * len(dl_train))

    best_acc   = 0.0
    best_state = None

    for ep in range(1, epochs + 1):
        net.train()
        pbar = tqdm(dl_train, desc=f"[seed={seed}] Epoch {ep:02d}/{epochs}",
                    leave=False)
        for x, y in pbar:
            x, y = x.to(device), y.to(device)
            opt.zero_grad()
            F.cross_entropy(net(x), y).backward()
            opt.step(); sched.step()

        acc = eval_accuracy(net, dl_test)
        print(f"Epoch {ep:02d}: val accuracy = {acc*100:.2f}%")
        if acc > best_acc:
            best_acc, best_state = acc, copy.deepcopy(net.state_dict())
            torch.save(
                {"epoch":  ep,
                 "seed":   seed,
                 "val_acc":acc,
                 "state_dict": best_state},
                MODELS_PATH / f"best_{seed}.pth")

    net.load_state_dict(best_state)
    net.eval()
    for p in net.parameters():
        p.requires_grad_(False)

    print(f"Лучшая accurasy на сиде {seed}: {best_acc*100:.2f}%\n")
    return net, best_acc

resnet_A, acc_A = train_resnet(seed=42)
resnet_B, acc_B = train_resnet(seed=99)

In [47]:
print(f"ResNet-A clean-acc: {acc_A*100:.2f}%")
print(f"ResNet-B clean-acc: {acc_B*100:.2f}%")
src_nets = [resnet_A, resnet_B]

ResNet-A clean-acc: 93.97%
ResNet-B clean-acc: 94.08%


In [59]:
def load_finetuned_resnet(ckpt_path: Path):
    ckpt = torch.load(ckpt_path, map_location=device)

    net = models.resnet18(weights=None)
    net.fc = nn.Linear(net.fc.in_features, 10)
    net.load_state_dict(ckpt["state_dict"])
    net.to(device).eval()
    for p in net.parameters():
        p.requires_grad_(False)

    print(f"Загружена модель из {ckpt_path.name},  val-acc = {ckpt['val_acc']*100:.2f}%")
    return net

seeds = [42, 99]
net_names = [f'best_{x}.pth' for x in seeds]
src_nets = [load_finetuned_resnet(MODELS_PATH / x) for x in net_names]

Загружена модель из best_42.pth,  val-acc = 93.97%
Загружена модель из best_99.pth,  val-acc = 94.08%


Создадим нашу основную модель, которую мы будем обучать

Последний (линейный) слой классификатора заменим, чтобы он предсказывал один из 10 классов (в предыдущей работе за нас это делал ultralytics, здесь же это приходится делать вручную)

Оптимизатор и планировщик так же раньше за нас добавляла библиотека, сейчас просто используем наиболее подходящие

In [60]:
from torch import nn
from ultralytics import YOLO

yolo = YOLO("yolov8s-cls.pt")
net  = yolo.model

head = net.model[-1]
in_f = head.linear.in_features
head.linear = nn.Linear(in_f, 10, bias=True)
net.to(device)

opt  = torch.optim.AdamW(net.parameters(), lr=BASE_LR, weight_decay=1e-4)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, EPOCHS*len(dl_train))

Напишем свой вариант FGSM-атаки. Очень хотелось бы использовать реализацию из ART, однако она может работать только на процессоре, и постоянные переводы с GPU на CPU существенно замедлят обучение

In [62]:
def get_logits(model, x):
    out = model(x)
    return out[0] if isinstance(out, (tuple, list)) else out

def fgsm(images, labels, src_model):
    eps_norm = torch.tensor(EPS, device=device) / torch.tensor(std, device=device).view(3,1,1)
    images = images.clone().detach().requires_grad_(True)

    logits = get_logits(src_model.eval(), images)
    loss = F.cross_entropy(logits, labels)
    loss.backward()

    adv = images + eps_norm * images.grad.sign()
    lo = (-torch.tensor(mean, device=device) / torch.tensor(std, device=device)).view(3,1,1)
    hi = ((1 - torch.tensor(mean, device=device)) / torch.tensor(std, device=device)).view(3,1,1)
    adv = adv.clamp(lo, hi).detach()
    return adv

Объявим ещё полезную функцию, которую будем вызывать каждые 10 эпох

In [57]:
def validate_and_checkpoint(
        epoch: int,
        net: torch.nn.Module,
        dl_test: torch.utils.data.DataLoader,
        ds_test_len: int,
        fgsm_fn,
        ckpt_dir: Path,
        opt: torch.optim.Optimizer,
        sched,
        best_robust: float,
        get_logits_fn,
        eps: float = 2 / 255
    ) -> float:
    net.eval()
    clean_hits = robust_hits = 0

    for x, y in dl_test:
        x, y = x.to(device), y.to(device)

        with torch.no_grad():
            clean_logits = get_logits_fn(net, x)

        adv_x = fgsm_fn(x, y, net)

        with torch.no_grad():
            adv_logits = get_logits_fn(net, adv_x)

        clean_hits  += (clean_logits.argmax(1) == y).sum().item()
        robust_hits += (adv_logits.argmax(1)  == y).sum().item()

    clean_acc  = clean_hits  / ds_test_len
    robust_acc = robust_hits / ds_test_len

    print(f"\nEpoch {epoch:02d}: "
          f"clean {clean_acc:.3f} | FGSM ε={eps} {robust_acc:.3f}")

    # ----------- сохранить обычный чекпоинт -----------
    ckpt_path = ckpt_dir / f"epoch_{epoch:02d}.pth"
    torch.save({
        "epoch": epoch,
        "model_state": net.state_dict(),
        "optimizer_state": opt.state_dict(),
        "scheduler_state": sched.state_dict(),
        "clean_acc": clean_acc,
        "robust_acc": robust_acc,
    }, ckpt_path)
    print(f"Saved checkpoint ➜ {ckpt_path.name}")

    # ----------- сохранить лучший ---------------------
    if robust_acc > best_robust:
        best_robust = robust_acc
        best_path   = ckpt_dir / "best.pth"
        torch.save(net.state_dict(), best_path)
        print(f"New best robust_acc = {best_robust:.3f} ➜ {best_path.name}")

    net.train()            # вернуться к обучению
    return best_robust

Наконец, можем переходить к обучению

In [70]:
from tqdm import tqdm

ckpt_dir = Path("checkpoints_eat")
ckpt_dir.mkdir(exist_ok=True, parents=True)

best_robust = 0.0

for epoch in range(1, EPOCHS + 1):
    net.train()
    running_acc = 0
    pbar = tqdm(dl_train, desc=f"Epoch {epoch:02d}/{EPOCHS}", leave=False)

    for step, (imgs, lbls) in enumerate(pbar, 1):
        imgs, lbls = imgs.to(device), lbls.to(device)

        # добавляем adv. examples
        src = random.choice(src_nets)
        k   = int(imgs.size(0) * ADV_FRACTION)
        adv = fgsm(imgs, lbls, src)

        mix_imgs   = torch.cat([imgs[k:], adv[:k]])
        mix_labels = torch.cat([lbls[k:], lbls[:k]])

        #  шаг оптимизации
        opt.zero_grad()
        logits = get_logits(net, mix_imgs)
        loss   = F.cross_entropy(logits, mix_labels, label_smoothing=0.1)
        loss.backward(); opt.step(); sched.step()

        # обновляем прогресс бар
        pred = logits.argmax(1)
        running_acc += (pred == mix_labels).sum().item()
        avg_acc = running_acc / (step * mix_labels.size(0))
        pbar.set_postfix(loss=f"{loss.item():.3f}",
                 clean_acc=f"{(pred[k:]==lbls[k:]).float().mean():.3f}",
                 mix_acc=f"{avg_acc:.3f}")

    if epoch % 10 == 0 or epoch == EPOCHS:
        best_robust = validate_and_checkpoint(
            epoch         = epoch,
            net           = net,
            dl_test       = dl_test,
            ds_test_len   = len(ds_test),
            fgsm_fn       = fgsm,
            ckpt_dir      = ckpt_dir,
            opt           = opt,
            sched         = sched,
            best_robust   = best_robust,
            get_logits_fn = get_logits,
            eps           = EPS
        )


Epoch 10: clean 0.347 | FGSM ε=0.00392156862745098 0.148
Saved checkpoint ➜ epoch_10.pth
New best robust_acc = 0.148 ➜ best.pth



Epoch 20: clean 0.348 | FGSM ε=0.00392156862745098 0.150
Saved checkpoint ➜ epoch_20.pth
New best robust_acc = 0.150 ➜ best.pth


KeyboardInterrupt: 

Доля правильных ответов на комбинированной выборке не превышает 34%, что делает эксперимент неудачным

В будушем можно к нему вернуться. Скорее всего, перед моделью была поставлена слишком тяжелая задача